In [1]:
import numpy as np
import pandas as pd
import torch

from utlis import train_test_split_data
from datasets import Dataset

from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    set_seed,
)

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

SEED = 42
set_seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if device.type == "cuda":
    torch.cuda.manual_seed_all(SEED)

Using device: cuda


In [3]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)

    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds),
        "mcc": matthews_corrcoef(labels, preds),
    }


### Loading and preparing Data

In [4]:
df = pd.read_pickle("../data/combined_df.pkl")
df["content_text_joined"] = df["content_words"].apply(lambda x: " ".join(x))
TEXT_COL = "content_text_joined"    
LABEL_COL = "is_spoiler"      

# Split
X_train, X_test, y_train, y_test = train_test_split_data(
    df,
    TEXT_COL,
    LABEL_COL
)

train_ds = Dataset.from_dict({
    "text": X_train,
    "labels": y_train.astype(int)
})

test_ds = Dataset.from_dict({
    "text": X_test,
    "labels": y_test.astype(int)
})

# validation split from train
split_ds = train_ds.train_test_split(test_size=0.2, seed=SEED)

### Training

In [5]:
MODEL_CHECKPOINT = "bert-base-uncased"
NUM_LABELS = 2

In [6]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_CHECKPOINT,
    num_labels=NUM_LABELS
)

model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [7]:
MAX_LENGTH = 256  # conscious trade-off

def tokenize(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        max_length=MAX_LENGTH
    )

tokenized_ds = {
    "train": split_ds["train"].map(tokenize, batched=True, remove_columns=["text"]),
    "val": split_ds["test"].map(tokenize, batched=True, remove_columns=["text"]),
}

data_collator = DataCollatorWithPadding(tokenizer)

Map:   0%|          | 0/56000 [00:00<?, ? examples/s]

Map:   0%|          | 0/14000 [00:00<?, ? examples/s]

In [8]:
# freeze all layers 
for param in model.bert.parameters():
    param.requires_grad = False

# unfreeze last 2 encoder layers
for layer in model.bert.encoder.layer[-2:]:
    for param in layer.parameters():
        param.requires_grad = True

# unfreeze classifier
for param in model.classifier.parameters():
    param.requires_grad = True


In [9]:
training_args = TrainingArguments(
    output_dir="./results/BERT",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,          # IMPORTANT: not 5
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="mcc",
    greater_is_better=True,
    logging_dir="./logs/BERT",
    report_to="none",
    seed=SEED,
)

c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["val"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

C:\Users\User\AppData\Local\Temp\ipykernel_24504\2330921089.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [11]:
trainer.train()

  0%|          | 0/10500 [00:00<?, ?it/s]

{'loss': 0.5734, 'grad_norm': 3.089932918548584, 'learning_rate': 1.904761904761905e-05, 'epoch': 0.14}
{'loss': 0.5375, 'grad_norm': 2.49786114692688, 'learning_rate': 1.8095238095238097e-05, 'epoch': 0.29}
{'loss': 0.5363, 'grad_norm': 5.688239574432373, 'learning_rate': 1.7142857142857142e-05, 'epoch': 0.43}
{'loss': 0.525, 'grad_norm': 2.8386781215667725, 'learning_rate': 1.6190476190476193e-05, 'epoch': 0.57}
{'loss': 0.5235, 'grad_norm': 4.357057094573975, 'learning_rate': 1.523809523809524e-05, 'epoch': 0.71}
{'loss': 0.5199, 'grad_norm': 1.6465082168579102, 'learning_rate': 1.4285714285714287e-05, 'epoch': 0.86}
{'loss': 0.5162, 'grad_norm': 2.41534161567688, 'learning_rate': 1.3333333333333333e-05, 'epoch': 1.0}


  0%|          | 0/875 [00:00<?, ?it/s]

{'eval_loss': 0.4939449727535248, 'eval_accuracy': 0.759, 'eval_f1': 0.6638102829812674, 'eval_mcc': 0.4780023292705823, 'eval_runtime': 213.1387, 'eval_samples_per_second': 65.685, 'eval_steps_per_second': 4.105, 'epoch': 1.0}
{'loss': 0.5135, 'grad_norm': 1.8786665201187134, 'learning_rate': 1.2380952380952383e-05, 'epoch': 1.14}
{'loss': 0.5109, 'grad_norm': 3.470393657684326, 'learning_rate': 1.1428571428571429e-05, 'epoch': 1.29}
{'loss': 0.5106, 'grad_norm': 2.1336960792541504, 'learning_rate': 1.0476190476190477e-05, 'epoch': 1.43}
{'loss': 0.4941, 'grad_norm': 2.1090333461761475, 'learning_rate': 9.523809523809525e-06, 'epoch': 1.57}
{'loss': 0.4988, 'grad_norm': 3.03007173538208, 'learning_rate': 8.571428571428571e-06, 'epoch': 1.71}
{'loss': 0.4953, 'grad_norm': 3.2108983993530273, 'learning_rate': 7.61904761904762e-06, 'epoch': 1.86}
{'loss': 0.5015, 'grad_norm': 2.1351068019866943, 'learning_rate': 6.666666666666667e-06, 'epoch': 2.0}


  0%|          | 0/875 [00:00<?, ?it/s]

{'eval_loss': 0.4868736267089844, 'eval_accuracy': 0.7653571428571428, 'eval_f1': 0.6885960754573893, 'eval_mcc': 0.5003628841705118, 'eval_runtime': 212.3609, 'eval_samples_per_second': 65.925, 'eval_steps_per_second': 4.12, 'epoch': 2.0}
{'loss': 0.498, 'grad_norm': 2.1290249824523926, 'learning_rate': 5.7142857142857145e-06, 'epoch': 2.14}
{'loss': 0.4826, 'grad_norm': 3.38681697845459, 'learning_rate': 4.761904761904762e-06, 'epoch': 2.29}
{'loss': 0.4859, 'grad_norm': 2.019333839416504, 'learning_rate': 3.80952380952381e-06, 'epoch': 2.43}
{'loss': 0.491, 'grad_norm': 3.456226110458374, 'learning_rate': 2.8571428571428573e-06, 'epoch': 2.57}
{'loss': 0.4815, 'grad_norm': 2.5250461101531982, 'learning_rate': 1.904761904761905e-06, 'epoch': 2.71}
{'loss': 0.4873, 'grad_norm': 2.8609414100646973, 'learning_rate': 9.523809523809525e-07, 'epoch': 2.86}
{'loss': 0.4842, 'grad_norm': 2.3618741035461426, 'learning_rate': 0.0, 'epoch': 3.0}


  0%|          | 0/875 [00:00<?, ?it/s]

{'eval_loss': 0.4820748567581177, 'eval_accuracy': 0.7712142857142857, 'eval_f1': 0.680753513405761, 'eval_mcc': 0.5045606600593938, 'eval_runtime': 213.4058, 'eval_samples_per_second': 65.603, 'eval_steps_per_second': 4.1, 'epoch': 3.0}
{'train_runtime': 4085.1996, 'train_samples_per_second': 41.124, 'train_steps_per_second': 2.57, 'train_loss': 0.5079563525971912, 'epoch': 3.0}


TrainOutput(global_step=10500, training_loss=0.5079563525971912, metrics={'train_runtime': 4085.1996, 'train_samples_per_second': 41.124, 'train_steps_per_second': 2.57, 'total_flos': 2.19859708969056e+16, 'train_loss': 0.5079563525971912, 'epoch': 3.0})

In [12]:
eval_results = trainer.evaluate()
print("Final evaluation:", eval_results)

  0%|          | 0/875 [00:00<?, ?it/s]

Final evaluation: {'eval_loss': 0.4820748567581177, 'eval_accuracy': 0.7712142857142857, 'eval_f1': 0.680753513405761, 'eval_mcc': 0.5045606600593938, 'eval_runtime': 211.2876, 'eval_samples_per_second': 66.26, 'eval_steps_per_second': 4.141, 'epoch': 3.0}


In [14]:
trainer.save_model("../Models/BERTa/final_model")
tokenizer.save_pretrained("../Models/BERTa/tokenizer")

('../Models/BERTa/tokenizer\\tokenizer_config.json',
 '../Models/BERTa/tokenizer\\special_tokens_map.json',
 '../Models/BERTa/tokenizer\\vocab.txt',
 '../Models/BERTa/tokenizer\\added_tokens.json',
 '../Models/BERTa/tokenizer\\tokenizer.json')